In [2]:
import os
from langchain.llms import GooglePalm
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')
# api_key = os.getenv('OPENAI_API_KEY')
llm = GooglePalm(google_api_key=api_key, temperature=0.5)
# llm = OpenAI(openai_api_key=api_key, temperature=0.7)

In [3]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='af_tech_faqs.csv', source_column="prompt", encoding='ISO-8859-1')
data = loader.load()
# print(data)

## Embedded 
(Instruct Embeddings)

In [4]:


embeddings = OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings


OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000020DD6AB8AD0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000020DD6AC0E50>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [5]:

vector_db = FAISS.from_documents(documents=data, embedding=embeddings)


In [6]:
retriever = vector_db.as_retriever()
res = retriever.invoke("can you help me for finding jobs?")
print(res)

[Document(metadata={'source': 'Do you provide any job assistance?', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'), Document(metadata={'source': 'Will this course guarantee me a job?', 'row': 33}, page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless pri

In [7]:
from langchain.prompts import PromptTemplate
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer, just say "I dont know.".

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [8]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm, 
                                    chain_type="stuff",
                                    retriever=retriever,
                                    input_key="query",
                                    return_source_documents=True,
                                    chain_type_kwargs=chain_type_kwargs
                                    )

In [9]:
chain.invoke("do you have free course?")

{'query': 'do you have free course?',
 'result': 'response: Yes we have a much lighter version of this course on YouTube available for free (click this link)',
 'source_documents': [Document(metadata={'source': 'Will this course guarantee me a job?', 'row': 33}, page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.'),
  Document(metadata={'source': 'I\x92m not sure if this course is good enough for me to invest some money. What ca